In [4]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [5]:
from langchain_openai import AzureChatOpenAI

deployment_name = "leitura_documento_gpt4o"
openai_api_version="2023-05-15"
model = "gpt-4o"
temperature = 0

llm = AzureChatOpenAI(azure_deployment=deployment_name, openai_api_version=openai_api_version, temperature=temperature)

In [6]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_with_schema = llm.with_structured_output(Joke)

chain = prompt | llm_with_schema

chain.invoke({"query": joke_query})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')